# 🧪 Lab 2: Use an MCP Server as a Plugin in a Semantic Kernel Agent

In this lab, you'll learn how to **extend a Semantic Kernel agent** by connecting it to an **external MCP server**. MCP (Model Context Protocol) allows agents to invoke external tools, services, or other agents as plugins.

You'll specifically:
- Connect to the **GitHub MCP server** as a tool via `MCPStreamableHttpPlugin`
- Use this plugin inside a **Semantic Kernel agent**
- Interact with the agent through a chat interface
- See the agent automatically call functions on the GitHub MCP server to answer questions

This lab showcases how Semantic Kernel can leverage **modular, tool-augmented AI workflows** by treating external MCP servers as powerful extensions to the agent's reasoning capabilities.


In [18]:
import os

from openai import AsyncAzureOpenAI
from dotenv import load_dotenv

from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

# Setup
load_dotenv(override=True)

True

In [19]:
# Check environment variables
print("AZURE_OPENAI_ENDPOINT:", os.getenv("AZURE_OPENAI_ENDPOINT"))
print("AZURE_OPENAI_KEY:", "***" if os.getenv("AZURE_OPENAI_KEY") else "Not set")
print("MODEL_DEPLOYMENT_NAME:", os.getenv("MODEL_DEPLOYMENT_NAME"))
print("MODEL_DEPLOYMENT_API_VERSION:", os.getenv("MODEL_DEPLOYMENT_API_VERSION"))

AZURE_OPENAI_ENDPOINT: https://sunhu-mhjykg2a-swedencentral.cognitiveservices.azure.com/openai/responses?api-version=2025-04-01-preview
AZURE_OPENAI_KEY: ***
MODEL_DEPLOYMENT_NAME: gpt-5-mini
MODEL_DEPLOYMENT_API_VERSION: 2025-04-01-preview


The code block below initializes and connects to a GitHub MCP (Model Context Protocol) plugin using the `MCPSsePlugin` interface.

- `MCPSsePlugin` 

In [20]:
codebeamer_plugin = MCPStreamableHttpPlugin(
    name="Codebeamer",
    description="Codebeamer Plugin",
    url="http://localhost:8080/mcp",
)

# Start the connection to the MCP plugin
await codebeamer_plugin.connect()

Now, let's set up an AI agent that can interact with an external MCP server as a plugin.

We’ll first configure the Azure OpenAI chat completion service, then define an agent by giving it a name, description, and a clear set of instructions. 

Finally, we attach the MCP-based GitHub plugin so the agent can call functions exposed by the plugin to fulfill user requests dynamically.


In [21]:
# Create the chat completion service
service_id = "azure_openai_chat"
async_openai_client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("MODEL_DEPLOYMENT_API_VERSION"),
)

chat_service = AzureChatCompletion(
    service_id=service_id,
    async_client=async_openai_client,
    deployment_name=os.getenv("MODEL_DEPLOYMENT_NAME"),
)
settings = AzureChatPromptExecutionSettings(service_id=service_id)
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create the agent with the chat service and plugin
codebeamer_agent = ChatCompletionAgent(
    service=chat_service,
    name="CodebeamerAgent",
    description="A chat bot that helps users interact with Codebeamer.",
    instructions="""
You are a chat bot. And you help users interact with Codebeamer.
You are especially good at answering questions about the Microsoft semantic-kernel project.
You can call functions to get the information you need.
""",
    plugins=[codebeamer_plugin],
)

Now let's try it out! 🎉

We're spinning up a simple chat loop where you can talk directly with the agent — powered by Azure OpenAI and enhanced with GitHub superpowers via the MCP plugin.

Let’s see this agent in action! 🤖💬


In [23]:
# Create the thread
thread: ChatHistoryAgentThread = ChatHistoryAgentThread()

# Run the agent as a chat
while True:
    user_input = input("Enter your message:")
    print(f"\nYou: {user_input}")

    if user_input.lower() in ["exit", "quit"]:
        break

    response = await codebeamer_agent.get_response(
        input=user_input,
        thread=thread,
    )

    assistant_reply = str(response)
    print(f"Bot: {assistant_reply}")

await codebeamer_plugin.close()


You: hello


ServiceResponseException: ("<class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service failed to complete the prompt", BadRequestError('Error code: 400 - {\'error\': {\'message\': "Unsupported parameter: \'messages\'. In the Responses API, this parameter has moved to \'input\'. Try again with the new parameter. See the API documentation for more information: https://platform.openai.com/docs/api-reference/responses/create.", \'type\': \'invalid_request_error\', \'param\': None, \'code\': \'unsupported_parameter\'}}'))

### 🧪 Try it out!

In this lab, you used the GitHub MCP Server — but the real power comes from the flexibility of the `MCPStdioPlugin` in Semantic Kernel. This component allows you to connect to **any tool that implements the Model Context Protocol (MCP)**.

You can easily swap in other MCP-compatible servers, or even chain multiple plugins together to create powerful tool-augmented agents.

---

🧰 **Explore These MCP Plugin Servers:**

- 🔗 [Official MCP Server Integrations (GitHub)](https://github.com/modelcontextprotocol/servers?tab=readme-ov-file#%EF%B8%8F-official-integrations)  
  A growing list of plugins including GitHub, Weather, Jira, Open Interpreter, and more.

- 📚 [10 Must-Know MCP Servers for Developers (DevShorts)](https://www.devshorts.in/p/ten-must-know-mcp-servers-for-every?utm_source=chatgpt.com)  
  A curated blog post with descriptions, commands, and usage tips.
